In [88]:
pip install ratelimiter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [89]:
import pandas as pd
import numpy as np
import requests
from time import sleep
import datetime
from ratelimiter import RateLimiter
from google.colab import drive

In [92]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [91]:
#creation de la liste earoports europe dans lesquelles lufthansa opère
airports = pd.read_csv("airports.csv")
airports["Continent"] = airports["TimeZoneId"].apply(lambda x : x.split("/")[0])
europe = airports[airports['Continent']== "Europe"].reset_index(drop=True)["AirportCode"]

In [93]:
#creation des listes individuels 
np.random.seed(123)

ousmane , abdoul , abdel = np.array_split(np.random.permutation(europe), 3)

In [97]:
#collecte de flight_information_departures
def get_flight_information_departures(origins, date, access_tokens):
    flights = []

    rate_limiter = RateLimiter(max_calls=4, period=1)  # 4 requêtes par seconde
    api_key_index = 0
    requests_count = 0

    for origin in origins:
        for hour in range(24):
            for minute in ["00", "30"]:
                with rate_limiter:
                    if requests_count >= 550:
                        print(f"Changement de clé API : utilisation de la clé {api_key_index + 1}") 
                        api_key_index += 1
                        requests_count = 0
                        if api_key_index >= len(access_tokens):
                            print("Toutes les clés API ont atteint la limite de requêtes.")
                            return pd.DataFrame.from_dict(flights).drop_duplicates().reset_index(drop=True)

                    time_str = f"{hour:02d}:{minute}"
                    datetime = f"{date}T{time_str}"
                    
                    authentification = {"Authorization": "Bearer " + access_tokens[api_key_index]}

                    try:
                        response = requests.get(
                            ("https://api.lufthansa.com/v1/operations/customerflightinformation/departures/{origin}/{datetime}"
                             "?limit=100&offset=0"
                            ).format(origin=origin, datetime=datetime),
                            headers=authentification).json()

                        for flight in response['FlightInformation']['Flights']['Flight']:
                            # Extraction des données du vol 
                            departure = flight.get('Departure', {})
                            arrival = flight.get('Arrival',{})
                            operatingCarrier = flight.get('OperatingCarrier')
                            status = flight.get('Status')


                            row = {
                                'DpAirportCode': departure.get('AirportCode'),
                                'DpScheduledDate': departure.get('Scheduled', {}).get('Date'),
                                'DpScheduledTime': departure.get('Scheduled', {}).get('Time'),
                                'DpActualDate': departure.get('Actual', {}).get('Date'),
                                'DpActualTime': departure.get('Actual', {}).get('Time'),
                                'DpTerminalName': departure.get('Terminal', {}).get('Name'),
                                'DpTerminalGate': departure.get('Terminal', {}).get('Gate'),
                                'DpStatusCode': departure.get('Status', {}).get('Code'),
                                'DPStatusDescription': departure.get('Status', {}).get('Description'),
                                'AirlineID': operatingCarrier.get('AirlineID'),
                                'FlightNumber': operatingCarrier.get('FlightNumber'),
                                'ArrActualDate':arrival.get('Actual', {}).get('Date'),
                                'ArrActualTime':arrival.get('Actual', {}).get('Time'),
                                'ArrTerminalName':arrival.get('Terminal', {}).get('Name'),
                                'ArrTerminalGate':arrival.get('Terminal', {}).get('Gate'),
                                'ArrStatusCode':arrival.get('Status', {}).get('Code'),
                                'ArrStatusDescription':arrival.get('Status', {}).get('Description'),
                                'StatusCode' : status.get('Code'),
                                'StatusDescription' : status.get('Description')

                            }

                            flights.append(row)

                        requests_count += 1

                    except Exception as e:
                        print(f"Error for origin {origin}, time {time_str}: {e}")

    flight_information_departures = pd.DataFrame.from_dict(flights).reset_index(drop = True)
    flight_information_departures = flight_information_departures.drop_duplicates().reset_index(drop = True)
    return flight_information_departures

access_tokens = ["cfrg5fh2baqgcsr38c96qayd", "fvzvx8ksbyrvgz4xnyzd8qtk",
                 "ekwaezvwas9suzbyajukxhmq", "byf52whj3fc2fefyc6zv3d4j",
                 "uk6c7bs6z6xtcyh2exx8r3aa", "7uweb7pf28sjug9sh8n33vf4",
                 "zex3j4ckfbh9rkwh9a8xcxv7", "y28tqg47n5ryqygyvxfxdy6a"]
origins = ["CDG"]
date = "2023-05-10"

flight_information_departures = get_flight_information_departures(origins, date, access_tokens)
flight_information_departures.to_csv("/content/drive/MyDrive/flight_information_departures_europe.csv", index=False)
display(flight_information_departures)


Error for origin CDG, time 00:00: 'FlightInformation'
Error for origin CDG, time 00:30: 'FlightInformation'
Error for origin CDG, time 01:00: 'FlightInformation'
Error for origin CDG, time 01:30: 'FlightInformation'
Error for origin CDG, time 02:00: 'str' object has no attribute 'get'
Error for origin CDG, time 02:30: 'str' object has no attribute 'get'
Error for origin CDG, time 20:30: 'str' object has no attribute 'get'
Error for origin CDG, time 21:00: 'FlightInformation'
Error for origin CDG, time 21:30: 'FlightInformation'
Error for origin CDG, time 22:00: 'FlightInformation'
Error for origin CDG, time 22:30: 'FlightInformation'
Error for origin CDG, time 23:00: 'FlightInformation'
Error for origin CDG, time 23:30: 'FlightInformation'


,DpAirportCode,DpScheduledDate,DpScheduledTime,DpActualDate,DpActualTime,DpTerminalName,DpTerminalGate,DpStatusCode,DPStatusDescription,AirlineID,FlightNumber,ArrActualDate,ArrActualTime,ArrTerminalName,ArrTerminalGate,ArrStatusCode,ArrStatusDescription,StatusCode,StatusDescription
0,CDG,2023-05-10,06:00,2023-05-10,06:00,1,061,DP,Flight Departed,LH,1053,2023-05-10,07:23,1,A28,LD,Flight Landed,LD,Flight Landed
1,CDG,2023-05-10,06:45,2023-05-10,06:41,1,065,DP,Flight Departed,LH,2239,2023-05-10,08:04,2,K12,LD,Flight Landed,LD,Flight Landed
2,CDG,2023-05-10,07:00,2023-05-10,07:16,1,063,DP,Flight Departed,BT,647,2023-05-10,08:36,None,None,LD,Flight Landed,LD,Flight Landed
3,CDG,2023-05-10,07:05,2023-05-10,07:07,1,068,DP,Flight Departed,OS,420,2023-05-10,08:51,3,None,LD,Flight Landed,LD,Flight Landed
4,CDG,2023-05-10,07:40,2023-05-10,08:11,1,067,DP,Flight Departed,LH,1051,2023-05-10,09:21,1,Area A,LD,Flight Landed,LD,Flight Landed
5,CDG,2023-05-10,08:30,2023-05-10,09:02,1,None,DP,Flight Departed,SN,3632,2023-05-10,09:55,None,None,LD,Flight Landed,LD,Flight Landed
6,CDG,2023-05-10,09:00,2023-05-10,09:16,1,065,DP,Flight Departed,LH,2227,2023-05-10,10:41,2,G10,LD,Flight Landed,LD,Flight Landed
7,CDG,2023-05-10,09:35,2023-05-10,09:47,1,064,DP,Flight Departed,LH,1027,2023-05-10,11:14,1,A15,LD,Flight Landed,LD,Flight Landed
8,CDG,2023-05-10,09:50,2023-05-10,10:10,1,068,DP,Flight Departed,LX,633,2023-05-10,11:11,None,None,LD,Flight Landed,LD,Flight Landed
9,CDG,2023-05-10,09:55,2023-05-10,10:31,1,None,DP,Flight Departed,SN,3634,2023-05-10,11:29,None,None,LD,Flight Landed,LD,Flight Landed


In [99]:
#collecte de flight_information_arrivals
def get_flight_information_arrivals(origins, date, access_tokens):
    flights = []

    rate_limiter = RateLimiter(max_calls=4, period=1)  # 4 requêtes par seconde
    api_key_index = 0
    requests_count = 0

    for origin in origins:
        for hour in range(24):
            for minute in ["00", "30"]:
                with rate_limiter:
                    if requests_count >= 750:
                        print(f"Changement de clé API : utilisation de la clé {api_key_index + 1}") 
                        api_key_index += 1
                        requests_count = 0
                        if api_key_index >= len(access_tokens):
                            print("Toutes les clés API ont atteint la limite de requêtes.")
                            return pd.DataFrame.from_dict(flights).drop_duplicates().reset_index(drop=True)

                    time_str = f"{hour:02d}:{minute}"
                    datetime = f"{date}T{time_str}"
                    
                    authentification = {"Authorization": "Bearer " + access_tokens[api_key_index]}

                    try:
                        response = requests.get(
                            ("https://api.lufthansa.com/v1/operations/customerflightinformation/arrivals/{origin}/{datetime}"
                             "?limit=100&offset=0"
                            ).format(origin=origin, datetime=datetime),
                            headers=authentification).json()

                        for flight in response['FlightInformation']['Flights']['Flight']:
                            # Extraction des données du vol
                            arrival = flight.get('Arrival',{})
                            departure = flight.get('Departure', {})
                            operatingCarrier = flight.get('OperatingCarrier')
                            status = flight.get('Status')

                            row = {
                                'ArrAirportCode': arrival.get('AirportCode'),
                                'ArrScheduledDate': arrival.get('Scheduled', {}).get('Date'),
                                'ArrScheduledTime': arrival.get('Scheduled', {}).get('Time'),
                                'ArrActualDate': arrival.get('Actual', {}).get('Date'),
                                'ArrActualTime': arrival.get('Actual', {}).get('Time'),
                                'ArrTerminalName': arrival.get('Terminal', {}).get('Name'),
                                'ArrTerminalGate': arrival.get('Terminal', {}).get('Gate'),
                                'ArrStatusCode': arrival.get('Status', {}).get('Code'),
                                'ArrStatusDescription': arrival.get('Status', {}).get('Description'),
                                'AirlineID': operatingCarrier.get('AirlineID'),
                                'FlightNumber': operatingCarrier.get('FlightNumber'),
                                'DpActualDate':departure.get('Actual', {}).get('Date'),
                                'DpActualTime':departure.get('Actual', {}).get('Time'),
                                'DpTerminalName':departure.get('Terminal', {}).get('Name'),
                                'DpTerminalGate':departure.get('Terminal', {}).get('Gate'),
                                'DpStatusCode':departure.get('Status', {}).get('Code'),
                                'DpStatusDescription':departure.get('Status', {}).get('Description'),
                                'StatusCode' : status.get('Code'),
                                'StatusDescription' : status.get('Description')

                            }
                            flights.append(row)

                        requests_count += 1

                    except Exception as e:
                        print(f"Error for origin {origin}, time {time_str}: {e}")

    flight_information_arrivals = pd.DataFrame.from_dict(flights).reset_index(drop = True)
    flight_information_arrivals = flight_information_arrivals.drop_duplicates().reset_index(drop = True)
    return flight_information_arrivals

access_tokens = ["cfrg5fh2baqgcsr38c96qayd", "apj5gpxj9c9ut9up6g68gsxs",
                 "5a5zsvd83ym6nb4mz47ddjds", "s7j65m93x5ap6ne7qfk4dkva",
                 "4hv9m2n4epq83z7j3g5j8ne8", "mucjtvh5vnbvuhpsktepesns",
                 "ztwk3etpxtvbnzmq5ex7vq8t", "fvzvx8ksbyrvgz4xnyzd8qtk",]

origins = ["CDG"]
date = "2023-05-10"

flight_information_arrivals = get_flight_information_arrivals(origins, date, access_tokens)
flight_information_arrivals.to_csv("/content/drive/MyDrive/flight_information_arrivals.csv", index=False)
display(flight_information_arrivals)

Error for origin CDG, time 00:00: 'FlightInformation'
Error for origin CDG, time 00:30: 'FlightInformation'
Error for origin CDG, time 01:00: 'FlightInformation'
Error for origin CDG, time 01:30: 'FlightInformation'
Error for origin CDG, time 02:00: 'FlightInformation'
Error for origin CDG, time 02:30: 'FlightInformation'
Error for origin CDG, time 03:00: 'FlightInformation'
Error for origin CDG, time 03:30: 'str' object has no attribute 'get'
Error for origin CDG, time 04:00: 'str' object has no attribute 'get'
Error for origin CDG, time 23:30: 'FlightInformation'


,ArrAirportCode,ArrScheduledDate,ArrScheduledTime,ArrActualDate,ArrActualTime,ArrTerminalName,ArrTerminalGate,ArrStatusCode,ArrStatusDescription,AirlineID,FlightNumber,DpActualDate,DpActualTime,DpTerminalName,DpTerminalGate,DpStatusCode,DpStatusDescription,StatusCode,StatusDescription
0,CDG,2023-05-10,07:30,2023-05-10,07:55,1,None,LD,Flight Landed,SN,3631,2023-05-10,06:59,None,None,DP,Flight Departed,LD,Flight Landed
1,CDG,2023-05-10,08:15,2023-05-10,08:19,1,067,LD,Flight Landed,LH,2226,2023-05-10,06:58,2,G44,DP,Flight Departed,LD,Flight Landed
2,CDG,2023-05-10,08:20,2023-05-10,08:36,1,064,LD,Flight Landed,LH,1026,2023-05-10,07:24,1,A52,DP,Flight Departed,LD,Flight Landed
3,CDG,2023-05-10,09:00,2023-05-10,09:00,1,068,LD,Flight Landed,LX,632,2023-05-10,07:43,None,None,DP,Flight Departed,LD,Flight Landed
4,CDG,2023-05-10,09:00,2023-05-10,09:27,1,None,LD,Flight Landed,SN,3633,2023-05-10,08:40,None,None,DP,Flight Departed,LD,Flight Landed
5,CDG,2023-05-10,09:10,2023-05-10,09:18,1,None,LD,Flight Landed,OS,411,2023-05-10,07:32,3,F25,DP,Flight Departed,LD,Flight Landed
6,CDG,2023-05-10,09:45,2023-05-10,09:50,1,067,LD,Flight Landed,LH,1028,2023-05-10,08:47,1,A38,DP,Flight Departed,LD,Flight Landed
7,CDG,2023-05-10,10:45,2023-05-10,11:04,1,064,LD,Flight Landed,LH,1030,2023-05-10,10:00,1,A25,DP,Flight Departed,LD,Flight Landed
8,CDG,2023-05-10,11:25,2023-05-10,11:41,1,065,LD,Flight Landed,LH,2228,2023-05-10,10:12,2,G46,DP,Flight Departed,LD,Flight Landed
9,CDG,2023-05-10,13:50,2023-05-10,14:23,1,065,LD,Flight Landed,LH,2230,2023-05-10,12:56,2,K22,DP,Flight Departed,LD,Flight Landed
